In [1]:
import json
import pandas as pd

In [4]:
import json
import pandas as pd

# edited 0404_0320
def get_test_annot_from_csv(
        base_root, csv_path, test_json_path, save_json_path, conf_thrs, area_min, area_max
    ):
    # [ps. annot이 존재하지 않는 test image를 json에서 제외하지는 않습니다.]
    # csv 파일의 annot을 반영하여 새로운 test json으로 저장하는 함수
    
    # base_root : 가장 기본이 되는 root
    # csv_path : 모델 예측 결과인 csv 파일의 path
    # test_json_path : 대회에서 제공된 test json의 path
    # save_json_path : annot이 추가된 새로운 test json을 저장하는 path
    # conf_thrs : confidence 기준
    # area_min, area_max : bbox area 기준 
    
    df = pd.read_csv(base_root + csv_path)
    with open(base_root + test_json_path, 'r') as f:
        test_json = json.loads(f.read()) # test json load

    annot_cnt = 0 # annot count를 annot_id로 활용
    num_image = len(df)
    for i in range(num_image):
        # df['image_id'][i] : test/0000.jpg
        # int : 0000 -> 0 변환
        img_id = int(str(df['image_id'][i]).split('/')[-1].split('.')[0])
        bbox_info = str(df['PredictionString'][i]).split(' ')[:-1]
        
        num_bbox = len(bbox_info)//6
        if num_bbox != 0: # 이미지 i에 bbox가 존재하면
            for bbox_id in range(num_bbox):
                # coco format에 필요한 annot 정보
                categoryid = int(bbox_info[6*bbox_id+0])
                confidence = float(bbox_info[6*bbox_id+1])
                x = float(bbox_info[6*bbox_id+2]) # bbox x
                y = float(bbox_info[6*bbox_id+3]) # bbox y
                w = float(bbox_info[6*bbox_id+4]) - float(bbox_info[6*bbox_id+2]) # bbox width
                h = float(bbox_info[6*bbox_id+5]) - float(bbox_info[6*bbox_id+3]) # bbox height
                
                # bbox가 confidence, bbox area 기준을 충족한다면
                if confidence>conf_thrs and area_min<w*h and w*h<area_max:
                    annot_dict = dict(
                        image_id=img_id,        # image id
                        category_id=categoryid, # bbox class
                        area=w*h,               # bbox area
                        bbox=[x, y, w, h],      # coco format
                        iscrowd=0,              # train.josn에서 모두 0
                        id=annot_cnt            # annot id
                    )
                    annot_cnt += 1              # annot count update
                    test_json['annotations'].append(annot_dict)
    
    print(f"added annotation num is {annot_cnt}")

    # annot 정보를 추가한 test_json을 base_root + save_json_path에 save
    with open(base_root + save_json_path, 'w') as f:
        json.dump(test_json, f, indent=4)
    print(f"save on {base_root + save_json_path}")

# Example
csv_file = 'submission_epoch_20'
conf_thrs = 30
area_min = 0
area_percent_max = 100
get_test_annot_from_csv(
    base_root='/opt/ml/detection/',
    csv_path=f'work_dirs/exp37/Fold3/{csv_file}.csv',
    test_json_path='dataset/test.json',
    save_json_path=f'dataset/test_{csv_file}_conf{conf_thrs}.json',
    conf_thrs=conf_thrs*0.01,
    area_min=area_min*0.01,
    area_max=area_percent_max*0.01*1024*1024)

added annotation num is 23884
save on /opt/ml/detection/dataset/test_submission_epoch_20_conf30.json


In [6]:
import json
import pandas as pd

# edited 0404_0320
def combine_train_with_pseudo(
        base_root, train_json_path, test_json_path, save_json_path
    ):
    # [ps. pseudo-labeling된 test + train 데이터로 학습하고 싶다면
    #      위 함수의 결과로 얻은 test_json을 이 함수에 넣으면 됩니다.]
    # 위에서 만든 pseudo-labeling된 test 데이터를 train json에 합치는 함수
    
    # base_root : 가장 기본이 되는 root
    # train_json_path : train json path 또는 cv_train json path (자유)
    # test_json_path : pseudo-laebling이 추가된 test json path (중요: 위 함수의 결과)
    # save_json_path : test data를 추가한 새로운 train json을 저장하는 path (자유)
    
    with open(base_root + train_json_path, 'r') as f:
        train_json = json.loads(f.read()) # train json load
    with open(base_root + test_json_path, 'r') as f:
        test_json = json.loads(f.read())  # pseudo-labeling이 추가된 test json load

    print(f"before image num is {len(train_json['images'])}")
    print(f"before annot num is {len(train_json['annotations'])}")
    print(f"before image-annot ratio is {len(train_json['annotations'])/len(train_json['images'])}")

    # with_annot : annot이 존재하는 image를 구분하기 위함
    with_annot = list(set([annot['image_id'] for annot in test_json['annotations']]))
    
    for image in test_json['images']:
        if image['id'] in with_annot:                        # image가 annot을 포함한다면
            save_id = image['id']                            # 추후 비교를 위해 test_json의 image id 저장
            image['id'] = train_json['images'][-1]['id'] + 1 # train_json의 마지막 image id에 1을 더함
            train_json['images'].append(image)               # train_json의 image list에 test image 추가
    
            for annot in test_json['annotations']:
                if annot['image_id'] == save_id:                          # annot의 image_id가 save_id와 같다면
                    annot['image_id'] = train_json['images'][-1]['id']    # train_json의 마지막 image id(앞서 1을 더함)를 annot의 image_id로 update
                    annot['id'] = train_json['annotations'][-1]['id'] + 1 # train_json의 마지막 annot id에 1을 더함
                    train_json['annotations'].append(annot)               # train_json의 annotations list에 test annotation 추가
    
    print(f"after image num is {len(train_json['images'])}")
    print(f"after annot num is {len(train_json['annotations'])}")
    print(f"after image-annot ratio is {len(train_json['annotations'])/len(train_json['images'])}")

    # test annot을 추가한 train_json을 base_root + save_json_path에 save
    with open(base_root + save_json_path, 'w') as f:
        json.dump(train_json, f, indent=4)
    print(f"save on {base_root + save_json_path}")

# Example
json_name = f'test_{csv_file}_conf{conf_thrs}' # csv_file, conf_thrs는 위의 cell에서 받아옵니다.
combine_train_with_pseudo(
    base_root='/opt/ml/detection/',
    train_json_path='dataset/stratified_kfold/train_3.json',
    test_json_path=f'dataset/{json_name}.json',
    save_json_path=f'dataset/stratified_kfold/train_3_{json_name}.json')

before image num is 3878
before annot num is 18398
before image-annot ratio is 4.744198040226921
after image num is 8749
after annot num is 42282
after image-annot ratio is 4.832780889244485
save on /opt/ml/detection/dataset/stratified_kfold/train_3_test_submission_epoch_20_conf30.json
